In [1]:
from diffop_factorization import InvSub

## Generic examples

In [2]:
mat1 = MatrixSpace(CC, 3).random_element().change_ring(CBF)
mat2 = MatrixSpace(CC, 3).random_element().change_ring(CBF)

(InvSub([mat1]), InvSub([mat1, mat2]))

([([1.000000 +/- 1.73e-7] + [+/- 1.73e-7]*I, [0.895570 +/- 5.16e-7] + [1.324287 +/- 5.74e-7]*I, [-0.832800 +/- 5.42e-7] + [0.379338 +/- 3.33e-7]*I)],
 None)

## Two matrices with a commun eigenvector

In [3]:
C = ComplexBallField(100)
mat1 = MatrixSpace(CC, 3).random_element().change_ring(C)
mat2 = MatrixSpace(CC, 3).random_element().change_ring(C)
mat1[1:3,0] , mat2[1:3,0] = MatrixSpace(C, 2, 1).zero(), MatrixSpace(C, 2, 1).zero()
ran = MatrixSpace(CC, 3).random_element().change_ring(C)
mat1, mat2 = ~ran * mat1 * ran, ~ran * mat2 * ran

V = InvSub([mat1, mat2]); V

[([-0.237510655694894 +/- 4.21e-16] + [0.155856535417258 +/- 4.74e-16]*I, [0.023976475679145 +/- 5.27e-16] + [-0.218089583640285 +/- 4.65e-16]*I, [1.000000000000000 +/- 7.61e-16] + [+/- 7.61e-16]*I)]

In [4]:
v = V[0]
v1, v2 = mat1*v, mat2*v
v/v[0], v1/v1[0], v2/v2[0]

(([1.0000000000000 +/- 4.47e-15] + [+/- 4.47e-15]*I, [-0.49174833943997 +/- 5.37e-15] + [0.59554124316715 +/- 7.16e-15]*I, [-2.9430370329053 +/- 1.86e-14] + [-1.9312462180330 +/- 2.65e-14]*I),
 ([1.000000000000 +/- 1.34e-14] + [+/- 1.34e-14]*I, [-0.4917483394400 +/- 4.40e-14] + [0.5955412431671 +/- 6.22e-14]*I, [-2.9430370329053 +/- 5.14e-14] + [-1.9312462180330 +/- 5.93e-14]*I),
 ([1.000000000000 +/- 1.08e-14] + [+/- 1.08e-14]*I, [-0.4917483394400 +/- 3.62e-14] + [0.5955412431671 +/- 5.45e-14]*I, [-2.9430370329053 +/- 3.80e-14] + [-1.9312462180330 +/- 4.59e-14]*I))

## Matrices with a prescribed dimension of invariant subspace

In [5]:
def test (td, pd, prec, N) :
    
    """ td = total dimension,
        pd = prescribed dimension,
        prec = precision,
        N = number of matrices """

    C = ComplexBallField(prec)
    
    Mats = []
    for cpt in range(N):
        mat = MatrixSpace(CC, td).random_element().change_ring(C)
        mat[pd:,:pd] = MatrixSpace(C, td-pd, pd).zero()
        Mats.append(mat)
    
    ran = MatrixSpace(CC, td).random_element().change_ring(C)
    for j, mat in enumerate(Mats):
        Mats[j] = ~ran * mat * ran
    
    V = InvSub(Mats)
    if V is None:
        print('cc')
    return pd == len(V)

dim_max = 20
%time all(test(dim, dim//3, 1000, 5) for dim in [4,6..dim_max]) # 10sec for dim_max = 20

CPU times: user 10.1 s, sys: 0 ns, total: 10.1 s
Wall time: 10.1 s


True

## Tricky case of MeatAxe algorithm

In [6]:
bN = 2         # number of blocs
bdim = 3       # dimension of blocs
N = bdim**2    # number of matrices
dim = bdim*bN  # total dimension
prec = 500
C = ComplexBallField(prec)

Mats = []
for cpt in range(N):
    A = MatrixSpace(CC, bdim).random_element().change_ring(C)
    mat = block_matrix(C, [[A.parent().zero()]*i+[A]+[A.parent().random_element()\
                                                      for j in range(bN-i-1)] for i in range(bN)])
    Mats.append(mat)

ran = MatrixSpace(CC, dim).random_element().change_ring(C)
Mats = [~ran * mat * ran for mat in Mats]

V = InvSub(Mats)
len(V)

3

## When a linear combination is not enough

In [7]:
prec = 100
C = ComplexBallField(prec)

mat1 = matrix(C, [[0,1,0],[0,0,1],[0,0,0]])
mat2 = matrix(C, [[0,0,0],[1,0,0],[0,-1,0]])
ran = MatrixSpace(CC, 3).random_element().change_ring(C)
mat1, mat2 = ~ran * mat1 * ran, ~ran * mat2 * ran

InvSub([mat1, mat2], verbose=True) is None

The partition is currently:  [3]
Lines checked
Need to check nolines
Check in a subspace of dimension 3
Need to compute a basis of the algebra


True